In [1]:
import random
import numpy as np
import time

class Celula:
    def __init__(self):
        self.tem_abismo = False
        self.tem_wumpus = False
        self.tem_ouro = False
        self.tem_brisa = False
        self.tem_fedor = False

class MundoWumpus:
    def __init__(self, tamanho=4):
        self.tamanho = tamanho
        self.agente_e_bot = True
        self.posicoes_abismos = [(3, 1), (4, 4), (1, 3)]
        self.wumpus_pos_inicial = (3, 4)
        self.ouro_pos_inicial = (2, 3)
        self.grade = {}
        self.reiniciar()

    def _obter_tupla_estado_atual(self):
        return (self.pos_agente, self.ouro_foi_pego, self.wumpus_esta_vivo)

    def exibir_mapa(self):
        print('🪨' * (self.tamanho + 2))
        for y in range(self.tamanho, 0, -1):
            linha = '🪨'
            for x in range(1, self.tamanho + 1):
                pos = (x, y)
                if pos == self.pos_agente:
                    linha += '🤖'
                elif self.grade[pos].tem_abismo:
                    linha += '🔳'
                elif pos == self.pos_wumpus and self.wumpus_esta_vivo:
                    linha += '🐸'
                elif pos == self.pos_wumpus and not self.wumpus_esta_vivo:
                    linha += '💀'
                elif pos == self.pos_ouro and not self.ouro_foi_pego:
                    linha += '💰'
                else:
                    linha += '▫️'
            linha += '🪨'
            print(linha)
        print('🪨' * (self.tamanho + 2))

    def atualizar_percepcoes(self):
        for celula in self.grade.values():
            celula.tem_brisa = celula.tem_fedor = False

        for pos, celula in self.grade.items():
            if celula.tem_abismo:
                for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                    adjacente = (pos[0] + dx, pos[1] + dy)
                    if adjacente in self.grade:
                        self.grade[adjacente].tem_brisa = True

        if self.wumpus_esta_vivo:
            for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                adjacente = (self.pos_wumpus[0] + dx, self.pos_wumpus[1] + dy)
                if adjacente in self.grade:
                    self.grade[adjacente].tem_fedor = True

    def reiniciar(self):
        self.pos_agente = (1, 1)
        self.ouro_foi_pego = False
        self.wumpus_esta_vivo = True
        self.pos_wumpus = self.wumpus_pos_inicial
        self.pos_ouro = self.ouro_pos_inicial
        self.flecha_disponivel = True
        self.pontuacao = 0

        self.grade = {(x, y): Celula() for x in range(1, self.tamanho + 1) for y in range(1, self.tamanho + 1)}
        for pos in self.posicoes_abismos:
            self.grade[pos].tem_abismo = True
        self.grade[self.pos_wumpus].tem_wumpus = True
        self.grade[self.pos_ouro].tem_ouro = True

        self.atualizar_percepcoes()
        return self._obter_tupla_estado_atual()

    def executar_passo(self, acao):
        recompensa = -1
        terminou = False
        info = ""

        if acao < 4:
            movimentos = {0: (0, 1), 1: (-1, 0), 2: (0, -1), 3: (1, 0)}
            dx, dy = movimentos[acao]
            new_pos = (self.pos_agente[0] + dx, self.pos_agente[1] + dy)

            if 1 <= new_pos[0] <= self.tamanho and 1 <= new_pos[1] <= self.tamanho:
                self.pos_agente = new_pos
                if self.grade[new_pos].tem_abismo or (new_pos == self.pos_wumpus and self.wumpus_esta_vivo):
                    recompensa = -1000
                    terminou = True
                    info = "Morreu"
                else:
                    info = f"Moveu para {new_pos}"
            else:
                info = "Bateu na parede"

        elif acao == 4:
            recompensa = -10
            if self.flecha_disponivel:
                self.flecha_disponivel = False
                for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                    alvo = (self.pos_agente[0] + dx, self.pos_agente[1] + dy)
                    if alvo == self.pos_wumpus:
                        self.wumpus_esta_vivo = False
                        self.atualizar_percepcoes()
                        recompensa += 100
                        info = "Matou o Wumpus"
                        break
                else:
                    info = "Flecha perdida"
            else:
                info = "Sem flechas"

        elif acao == 5:
            if self.pos_agente == self.pos_ouro and not self.ouro_foi_pego:
                self.ouro_foi_pego = True
                recompensa += 1000
                info = "Pegou o ouro"
            else:
                info = "Nada para pegar"

        if not self.wumpus_esta_vivo and self.ouro_foi_pego and self.pos_agente == (1, 1):
            recompensa += 1000
            terminou = True
            info = "Vitória!"

        self.pontuacao += recompensa
        if self.pontuacao < -150:
            terminou = True
            info = "Limite de passos excedido."

        return self._obter_tupla_estado_atual(), recompensa, terminou, info

class AgenteQLearning:
    def __init__(self, tamanho, acoes, taxa_aprendizado=0.1, fator_desconto=0.95, epsilon=0.9):
        self.tabela_q = np.zeros((tamanho + 1, tamanho + 1, 2, 2, acoes))
        self.taxa_aprendizado = taxa_aprendizado
        self.fator_desconto = fator_desconto
        self.epsilon = epsilon
        self.acoes = range(acoes)

    def _obter_indices_estado(self, tupla_estado):
        pos, tem_ouro, wumpus_vivo = tupla_estado
        return pos[0], pos[1], int(tem_ouro), int(wumpus_vivo)

    def escolher_acao(self, estado):
        if random.uniform(0, 1) < self.epsilon:
            return random.choice(self.acoes)
        else:
            indices_estado = self._obter_indices_estado(estado)
            return np.argmax(self.tabela_q[indices_estado])

    def aprender(self, estado, acao, recompensa, prox_estado):
        indices_estado = self._obter_indices_estado(estado)
        indices_prox_estado = self._obter_indices_estado(prox_estado)

        fatia_tabela_q_para_acao = indices_estado + (acao,)
        previsao = self.tabela_q[fatia_tabela_q_para_acao]

        recompensa_futura = np.max(self.tabela_q[indices_prox_estado])
        alvo = recompensa + self.fator_desconto * recompensa_futura

        self.tabela_q[fatia_tabela_q_para_acao] += self.taxa_aprendizado * (alvo - previsao)

def treinar_agente(episodios):
    env = MundoWumpus()
    agente = AgenteQLearning(tamanho=env.tamanho, acoes=6)
    recompensas = []

    taxa_decaimento_epsilon = 0.99995
    epsilon_minimo = 0.05

    for episodio in range(episodios):
        estado = env.reiniciar()
        terminou = False
        recompensa_total = 0

        while not terminou:
            acao = agente.escolher_acao(estado)
            prox_estado, recompensa, terminou, info = env.executar_passo(acao)
            agente.aprender(estado, acao, recompensa, prox_estado)
            estado = prox_estado
            recompensa_total += recompensa

        agente.epsilon = max(epsilon_minimo, agente.epsilon * taxa_decaimento_epsilon)
        recompensas.append(recompensa_total)
        if (episodio + 1) % 5000 == 0:
            recompensa_media = np.mean(recompensas[-5000:])
            print(f"Episódio {episodio + 1}/{episodios} - Recompensa Média: {recompensa_media:.2f} - Epsilon: {agente.epsilon:.3f}")

    return agente, env

if __name__ == '__main__':
    agente_treinado, env_treino = treinar_agente(episodios=100000)

    estado = env_treino.reiniciar()
    agente_treinado.epsilon = 0
    terminou = False
    print("-" * 20)

    recompensa_total_avaliacao = 0
    info = ""
    while not terminou:
        env_treino.exibir_mapa()
        pos, tem_ouro, wumpus_vivo = estado
        print(f"Último resultado: {info if info else 'Início'}")
        print("-" * 20)

        mapa_acoes = {0:'Cima', 1:'Esquerda', 2:'Baixo', 3:'Direita', 4:'Atirar', 5:'Pegar'}
        acao = agente_treinado.escolher_acao(estado)

        prox_estado, recompensa, terminou, info = env_treino.executar_passo(acao)
        recompensa_total_avaliacao += recompensa
        estado = prox_estado
        time.sleep(1)

    env_treino.exibir_mapa()
    print(f"Recompensa total na avaliação: {recompensa_total_avaliacao}")

Episódio 5000/100000 - Recompensa Média: -686.81 - Epsilon: 0.701
Episódio 10000/100000 - Recompensa Média: -307.81 - Epsilon: 0.546
Episódio 15000/100000 - Recompensa Média: 71.83 - Epsilon: 0.425
Episódio 20000/100000 - Recompensa Média: 426.51 - Epsilon: 0.331
Episódio 25000/100000 - Recompensa Média: 662.42 - Epsilon: 0.258
Episódio 30000/100000 - Recompensa Média: 900.51 - Epsilon: 0.201
Episódio 35000/100000 - Recompensa Média: 1079.03 - Epsilon: 0.156
Episódio 40000/100000 - Recompensa Média: 1253.27 - Epsilon: 0.122
Episódio 45000/100000 - Recompensa Média: 1457.92 - Epsilon: 0.095
Episódio 50000/100000 - Recompensa Média: 1578.00 - Epsilon: 0.074
Episódio 55000/100000 - Recompensa Média: 1638.57 - Epsilon: 0.058
Episódio 60000/100000 - Recompensa Média: 1714.76 - Epsilon: 0.050
Episódio 65000/100000 - Recompensa Média: 1731.75 - Epsilon: 0.050
Episódio 70000/100000 - Recompensa Média: 1843.60 - Epsilon: 0.050
Episódio 75000/100000 - Recompensa Média: 1732.09 - Epsilon: 0.050
E